MRI Dataset Cleaning

We will perform teh following steps to the data
Merge
Resize
Split and Save

Merging both datasets of Training and Test from the orignial data and randomly splitting after resizeing to ensure the data is randomised and properly representing the system without deviating from the orginal implementation methods of the paper

In [13]:
!pip install opencv-python

   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -- ------------------------------------- 2.4/39.0 MB 11.2 MB/s eta 0:00:04
   ---- ----------------------------------- 4.7/39.0 MB 11.4 MB/s eta 0:00:04
   ------ --------------------------------- 6.8/39.0 MB 11.3 MB/s eta 0:00:03
   --------- ------------------------------ 9.4/39.0 MB 11.5 MB/s eta 0:00:03
   ----------- ---------------------------- 11.5/39.0 MB 11.8 MB/s eta 0:00:03
   -------------- ------------------------- 14.4/39.0 MB 11.8 MB/s eta 0:00:03
   ----------------- ---------------------- 16.8/39.0 MB 11.7 MB/s eta 0:00:02
   ------------------- -------------------- 19.4/39.0 MB 11.9 MB/s eta 0:00:02
   ---------------------- ----------------- 22.3/39.0 MB 12.0 MB/s eta 0:00:02
   ------------------------- -------------- 24.6/39.0 MB 12.1 MB/s eta 0:00:02
   --------------------------- ------------ 27.3/39.0 MB 12.2 MB/s eta 0:00:01
   ------------------------------ --------- 29.9/39.0 MB 12.2 MB/

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.


In [17]:
import cv2
import numpy as np

print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.12 from "C:\Users\ekowd\anaconda3\python.exe"
  * The NumPy version is: "1.26.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


ImportError: numpy._core.multiarray failed to import

In [15]:
# Importing Libraries
import os
import shutil
from pathlib import Path
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.12 from "C:\Users\ekowd\anaconda3\python.exe"
  * The NumPy version is: "1.26.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


ImportError: numpy._core.multiarray failed to import

Since the raw data is already stored in the project repository, we'll call it from there rather than kaggle directly.

We will define the specific path for the raw data folders and the file path for the folder we will store the cleaned data

In [ ]:
# Define Paths
RAW_TRAIN_DIR = Path(r"C:\Users\ekowd\OneDrive\Desktop\Legon\final-year-project\data\raw_data\Training")
RAW_TEST_DIR  = Path(r"C:\Users\ekowd\OneDrive\Desktop\Legon\final-year-project\data\raw_data\Testing")
CLEAN_DIR     = Path(r"C:\Users\ekowd\OneDrive\Desktop\Legon\final-year-project\data\cleaned_data")

Next we will create teh folder structure in teh clean directory since we plan to split the data into train, test and validation datasets.

In [ ]:
# Configuration 
CLASSES = ['glioma', 'meningioma', 'notumor', 'pituitary']
IMG_SIZE = (224, 224)
SPLIT_RATIOS = {'train': 0.8, 'val': 0.1, 'test': 0.1}

# Create Folder Structure 
for split in ['train', 'val', 'test']:
    for cls in CLASSES:
        path = CLEAN_DIR / split / cls
        path.mkdir(parents=True, exist_ok=True)

print("Folder structure created under:", CLEAN_DIR)

Next we prepare the function to Load, Resize and collect the images, resizing them to 224x224 which provide the best results.

In [ ]:
# Function: Load, Resize, and Collect Image Paths
def load_and_resize_images(class_name):
    """
    Combines training and testing images for a given class,
    resizes to 224x224, and returns a list of (image_array, filename).
    """
    images = []
    # Combine both train and test directories
    for data_dir in [RAW_TRAIN_DIR, RAW_TEST_DIR]:
        class_dir = data_dir / class_name
        if not class_dir.exists():
            print(f" Skipped missing directory: {class_dir}")
            continue
        for img_file in class_dir.glob('*'):
            try:
                img = cv2.imread(str(img_file))
                if img is None:
                    continue
                img_resized = cv2.resize(img, IMG_SIZE)
                images.append((img_resized, img_file.name))
            except Exception as e:
                print(f"Error processing {img_file}: {e}")
    return images

Then we store the now resized images, passing it to our saving function

In [ ]:
# Function: Save Images 
def save_images(images, save_dir, class_name):
    """
    Saves a list of (image_array, filename) into a given directory.
    """
    for img_array, filename in images:
        save_path = save_dir / class_name / filename
        cv2.imwrite(str(save_path), img_array)

Now we porcess the classes for each type of tumor

In [ ]:
# Process Each Class
for cls in CLASSES:
    print(f"\n Processing class: {cls}")
    images = load_and_resize_images(cls)
    print(f"Total images found (merged): {len(images)}")

    # Prepare numpy arrays for splitting
    X = np.arange(len(images))  # dummy indices for split
    y = [cls] * len(images)

    # Split: train (80%) + temp (20%)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(1 - SPLIT_RATIOS['train']), random_state=42, shuffle=True
    )
    # Split temp into val (10%) and test (10%)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True
    )

    # Map indices to image data
    train_imgs = [images[i] for i in X_train]
    val_imgs   = [images[i] for i in X_val]
    test_imgs  = [images[i] for i in X_test]

    # Save into respective folders
    print("   Saving training images...")
    save_images(tqdm(train_imgs, leave=False), CLEAN_DIR / 'train', cls)

    print("   Saving validation images...")
    save_images(tqdm(val_imgs, leave=False), CLEAN_DIR / 'val', cls)

    print("   Saving testing images...")
    save_images(tqdm(test_imgs, leave=False), CLEAN_DIR / 'test', cls)

    print(f" Done: {cls} | Train: {len(train_imgs)} | Val: {len(val_imgs)} | Test: {len(test_imgs)}")

print("\n Dataset successfully cleaned, resized, and split!")